In [1]:
import sys, os
sys.path.append("..")
from common_utils import DATA_HOME
from lstm.sales_data import Sales_Dataset
from torch.utils.data.dataloader import DataLoader
import torch

In [2]:
SALE_HOME = os.path.join(DATA_HOME, "sales_data")

I, H, B = 71, 34, 3

sd = Sales_Dataset(SALE_HOME, seq_len=500)
test_set, train_set = torch.utils.data.random_split(sd, [0.2, 0.8])
train_dl = DataLoader(train_set, shuffle=True, batch_size=B)
test_dl = DataLoader(test_set, shuffle=True, batch_size=B)
len(train_dl), len(test_dl)

# # print(sd.O.describe())
# # print(sd.TR.describe())
# # print(sd.TS.describe())
# # print(sd.S.describe())
# print(sd[4][0].shape)
# print(sd[4][1].shape)
# # print(sd[4][0], sd[4][1])


(17107, 4277)

### PyTorch Model training

In [3]:
from torch.nn import LSTM
from torch.nn import MSELoss
from torch import optim
import matplotlib.pyplot as plt

model = LSTM(I, H, batch_first=True).cuda()
loss = MSELoss()
adam = optim.Adam(model.parameters(), lr=0.001)
h, c = torch.randn(1, B, H, dtype=torch.float32).cuda(), torch.randn(1, B, H, dtype=torch.float32).cuda()
tr_losses, inf_losses = [], []

train_gen = enumerate(train_dl)
test_gen = enumerate(test_dl)

for tr_idx, (X, y) in train_gen:
    tr_l, inf_l = 0, 0
    yhat, (_, _) = model(X)
    l = loss(yhat, y.cuda())

    adam.zero_grad()
    l.backward()
    adam.step()
    tr_l += l.item()

    try:
        inf_idx, (X, y) = next(test_gen)
    except StopIteration:
        test = enumerate(test_dl)
        inf_idx, (X, y) = next(test_gen)
    yhat, (_, _) = model(X)
    l = loss(yhat, y.cuda())
    inf_l += l.item()
    tr_losses.append(tr_l / len(train_dl))
    inf_losses.append(inf_l / len(test_dl))
    if tr_idx and tr_idx % 100 == 0:
        print(f"iteration: {tr_idx} train loss: {tr_l / len(train_dl)} inf loss: {inf_l / len(test_dl)}")
        torch.save(model.state_dict(), f"sales_model_{tr_idx}.pth")


iteration: 100 train loss: 4.791060858162503e-06 inf loss: 1.9026518230380257e-05
iteration: 200 train loss: 4.670148217894789e-06 inf loss: 1.870141893116592e-05
iteration: 300 train loss: 3.613832975009493e-06 inf loss: 1.7568979456738552e-05
iteration: 400 train loss: 4.660489944947026e-06 inf loss: 1.2709768974588719e-05
iteration: 500 train loss: 3.994695401076035e-06 inf loss: 1.916911227418241e-05
iteration: 600 train loss: 4.251345625373589e-06 inf loss: 9.953533206722964e-06
iteration: 700 train loss: 4.381447910266024e-06 inf loss: 1.7813078733367957e-05
iteration: 800 train loss: 3.82407244525494e-06 inf loss: 1.3594474466654751e-05
iteration: 900 train loss: 2.329859530033092e-06 inf loss: 1.4380350454627896e-05
iteration: 1000 train loss: 3.433919114261533e-06 inf loss: 1.4610778429755156e-05
iteration: 1100 train loss: 3.868455380095071e-06 inf loss: 1.6324445065254638e-05
iteration: 1200 train loss: 4.124427487014393e-06 inf loss: 1.411639138279269e-05
iteration: 1300 tr

### display losses

In [ ]:
plt.plot(range(len(tr_losses)), tr_losses, label="train loss")
plt.plot(range(len(inf_losses)), inf_losses, label="test loss")
plt.legend()
plt.show()


### Custom Model training

In [ ]:
from mylstm import MyLSTM
from torch.nn import MSELoss
from torch import optim
import matplotlib.pyplot as plt

I, H = 71, 33
epochs = 5

model = MyLSTM(71, 33).cuda()
loss = MSELoss()
# param yhat, y: 3d tensors
# def loss(yhat, y):
#     slen = yhat.shape[0] * yhat.shape[1] * yhat.shape[2]
#     return (((torch.log(yhat+1) - torch.log(y+1)) ** 2).sum() / slen) ** 0.5

adam = optim.Adam(model.parameters(), lr=0.001)
h, c = torch.randn(1, 1, 33, dtype=torch.float32).cuda(), torch.randn(1,1, 33, dtype=torch.float32).cuda()
tr_losses, inf_losses = [], []

train_gen = enumerate(train_dl)
test_gen = enumerate(test_dl)

for tr_idx, (X, y) in train_gen:
    tr_l, inf_l = 0, 0
    yhat, (_, _) = model(X, (h, c))
    l = loss(yhat, y.cuda())

    adam.zero_grad()
    l.backward()
    adam.step()
    tr_l += l.item()

    try:
        inf_idx, (X, y) = next(test_gen)
    except StopIteration:
        test = enumerate(test_dl)
        inf_idx, (X, y) = next(test_gen)
    yhat, (_, _) = model(X, (h, c))
    l = loss(yhat, y.cuda())
    inf_l += l.item()
    tr_losses.append(tr_l / len(train_dl))
    inf_losses.append(inf_l / len(test_dl))
    if tr_idx and tr_idx % 1000 == 0:
        print(f"iteration: {tr_idx} train loss: {tr_l /len(train_dl)} inf loss: {inf_l / len(test_dl)}")


In [ ]:
plt.plot(range(len(tr_losses)), tr_losses, label="train loss")
plt.plot(range(epochs), inf_losses, label="test loss")
plt.legend()
plt.show()
